In [ ]:
from transformers import AutoProcessor, Blip2ForConditionalGeneration
import torch
from glob import glob
import json
import os
from tqdm import tqdm

In [ ]:
processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16
).to("cuda")

In [ ]:
model.requires_grad_

In [ ]:
path = "/home/aihao/workspace/DeepLearningContent/datasets/images/data/pixiv/**"

In [ ]:
file_paths = glob(path, recursive=True)
file_paths

In [ ]:
save_path = os.path.join(
    "/home/aihao/workspace/DeepLearningContent/datasets/images/data/pixiv",
    "prompts.json",
)

In [ ]:
result = {}

In [ ]:

for file_path in tqdm(file_paths):
    try:
        image = Image.open(file_path)
    except IOError:
        continue
    else:
        inputs = processor(image, return_tensors="pt").to("cuda", torch.float16)
        generated_ids = model.generate(**inputs, max_new_tokens=20)
        generated_text = processor.batch_decode(
            generated_ids, skip_special_tokens=True
        )[0].strip()
        result[file_path] = generated_text

In [ ]:
result

In [ ]:
with open(save_path, "w") as f:
    json.dump(result, f)

In [ ]:
for k,v in result.items():
    print(k)
    print(v)
    break

In [ ]:
result2={}

In [ ]:
with open(save_path, "r") as f:
    result2=json.load(f)

In [ ]:
result2